In [ ]:
#Importing the libraries 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model


In [ ]:
# Loading the data 
cifar10 = tf.keras.datasets.cifar10

# Partition into Test and Train Data 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


170508288/170498071 [==============================] - 7s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [ ]:
x_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [ ]:
# Reducing pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# flattening the label values
y_train, y_test = y_train.flatten(), y_test.flatten()


In [ ]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
x_train[0]

array([[[0.23137255, 0.24313725, 0.24705882],
        [0.16862745, 0.18039216, 0.17647059],
        [0.19607843, 0.18823529, 0.16862745],
        ...,
        [0.61960784, 0.51764706, 0.42352941],
        [0.59607843, 0.49019608, 0.4       ],
        [0.58039216, 0.48627451, 0.40392157]],

       [[0.0627451 , 0.07843137, 0.07843137],
        [0.        , 0.        , 0.        ],
        [0.07058824, 0.03137255, 0.        ],
        ...,
        [0.48235294, 0.34509804, 0.21568627],
        [0.46666667, 0.3254902 , 0.19607843],
        [0.47843137, 0.34117647, 0.22352941]],

       [[0.09803922, 0.09411765, 0.08235294],
        [0.0627451 , 0.02745098, 0.        ],
        [0.19215686, 0.10588235, 0.03137255],
        ...,
        [0.4627451 , 0.32941176, 0.19607843],
        [0.47058824, 0.32941176, 0.19607843],
        [0.42745098, 0.28627451, 0.16470588]],

       ...,

       [[0.81568627, 0.66666667, 0.37647059],
        [0.78823529, 0.6       , 0.13333333],
        [0.77647059, 0

In [ ]:
# number of classes
K = len(set(y_train))

# calculate total number of classes
# for output layer
print("number of classes:", K)

# Build the model using the functional API
# input layer
i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.2)(x)

# Hidden layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)

# last hidden layer i.e.. output layer
x = Dense(K, activation='softmax')(x)

model = Model(i, x)

# model description
model.summary()


number of classes: 10
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16

In [ ]:
# Model Compile
model.compile(optimizer='adam',
			loss='sparse_categorical_crossentropy',
			metrics=['accuracy'])


In [ ]:
# Model Fitting
r = model.fit(
x_train, y_train, validation_data=(x_test, y_test), epochs=50)


Epoch 1/50
1563/1563 [==============================] - 45s 21ms/step - loss: 1.2906 - accuracy: 0.5533 - val_loss: 1.0673 - val_accuracy: 0.6253
Epoch 2/50
1563/1563 [==============================] - 31s 20ms/step - loss: 0.8471 - accuracy: 0.7054 - val_loss: 0.8731 - val_accuracy: 0.7035
Epoch 3/50
1563/1563 [==============================] - 31s 20ms/step - loss: 0.6903 - accuracy: 0.7604 - val_loss: 0.8154 - val_accuracy: 0.7268
Epoch 4/50
1563/1563 [==============================] - 30s 19ms/step - loss: 0.5805 - accuracy: 0.8002 - val_loss: 0.6740 - val_accuracy: 0.7729
Epoch 5/50
1563/1563 [==============================] - 30s 19ms/step - loss: 0.4933 - accuracy: 0.8309 - val_loss: 0.6211 - val_accuracy: 0.8016
Epoch 6/50
1563/1563 [==============================] - 30s 19ms/step - loss: 0.4125 - accuracy: 0.8580 - val_loss: 0.6081 - val_accuracy: 0.8039
Epoch 7/50
1563/1563 [==============================] - 30s 19ms/step - loss: 0.3614 - accuracy: 0.8757 - val_loss: 0.5958 -

In [ ]:
# Fit with data augmentation
batch_size = 32
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size

r = model.fit(train_generator, validation_data=(x_test, y_test),
			steps_per_epoch=steps_per_epoch, epochs=50)


Epoch 1/50
1562/1562 [==============================] - 55s 35ms/step - loss: 0.5468 - accuracy: 0.8272 - val_loss: 0.5005 - val_accuracy: 0.8346
Epoch 2/50
1562/1562 [==============================] - 53s 34ms/step - loss: 0.4331 - accuracy: 0.8575 - val_loss: 0.4748 - val_accuracy: 0.8437
Epoch 3/50
1562/1562 [==============================] - 53s 34ms/step - loss: 0.3999 - accuracy: 0.8680 - val_loss: 0.4528 - val_accuracy: 0.8532
Epoch 4/50
1562/1562 [==============================] - 53s 34ms/step - loss: 0.3768 - accuracy: 0.8753 - val_loss: 0.4507 - val_accuracy: 0.8485
Epoch 5/50
1562/1562 [==============================] - 53s 34ms/step - loss: 0.3644 - accuracy: 0.8778 - val_loss: 0.4760 - val_accuracy: 0.8437
Epoch 6/50
1562/1562 [==============================] - 53s 34ms/step - loss: 0.3497 - accuracy: 0.8812 - val_loss: 0.4751 - val_accuracy: 0.8478
Epoch 7/50
1562/1562 [==============================] - 53s 34ms/step - loss: 0.3326 - accuracy: 0.8871 - val_loss: 0.4313 -

In [ ]:
# label mapping

labels = ''''''.split()

# select the image from our test dataset
image_number = 0

# load the image in an array
n = np.array(x_train)

p=[]
# reshape it
for i in range(len(n)):
  p.append(n[i].reshape(1, 32, 32, 3))



In [ ]:
len(x_train)

50000

In [ ]:
predicted_label=[]
for i in range(50000):
 predicted_label.append(model.predict(p[i]).argmax())

In [ ]:
%pip install bloom-filter

In [ ]:
#Traditional Bloom Filter 
from bloom_filter import BloomFilter

In [ ]:
 def Train_overflow(bloom,train_features,y_new_train,preds,tau):
    X_train=train_features
    for i in range(len(preds)):
        if preds[i]>tau:
              if y_new_train[i] ==1:
                  bloom.add(str(X_train[i]))
    return bloom

In [ ]:
def binarized(Y_train):
  y=[]
  for i in range(len(Y_train)):
        if(Y_train[i]<=4):
          y.append(1)
        else:
          y.append(0)
  return y

In [ ]:
tau=4
bloom_traditional = BloomFilter(max_elements=25000)
y_new_train=binarized(y_train)

bloom_overflow=Train_overflow(bloom_traditional,x_train,y_new_train,predicted_label,tau)

In [ ]:
# loading the image in an array
n = np.array(x_test)
p=[]
# reshape it
for i in range(len(n)):
  p.append(n[i].reshape(1, 32, 32, 3))

predicted_tst=[]
for i in range(len(x_test)):
  predicted_tst.append(model.predict(p[i]).argmax())

In [ ]:
pred_new_test=binarized(predicted_tst)

In [ ]:
count=0
for i in range(len(predicted_tst)):
   if predicted_tst[i]!=y_test[i]:
     count+=1
print(count)

1254


In [ ]:
def positive_negative(X,Y,tau,predicted_tst):
    keys=[]
    non_keys=[]
    pred_keys=[]
    pred_non_keys=[]
    actual_for_keys=[]
    actual_for_non_keys=[]
    print("positive",len(Y))
    count1=0
    count2=0
    for i in range(len(predicted_tst)):
     if predicted_tst[i]<=tau:
        keys.append(X[i])
        pred_keys.append(1)
        if predicted_tst[i]!=Y[i]:
           actual_for_keys.append(Y[i])
           count1+=1
           #print(count1)
     elif str(X[i]) in bloom_overflow:
           keys.append(X[i])
           actual_for_keys.append([i,Y[i]])
           count2+=1
     else:
          non_keys.append(X[i])
          pred_non_keys.append(0)
          actual_for_non_keys.append([i,Y[i]])
           #print(count2)
    return keys,non_keys,pred_keys,pred_non_keys,actual_for_keys,actual_for_non_keys,count1,count2

In [ ]:
def Test_BF_FNR(model,bloom_overflow,non_keys,tau,prediction):
      output1=[]
      FN=0
      for i in range(len(non_keys)):
          if str(non_keys[i]) in bloom_overflow:
              output1.append(1)
              FN+=1
          else:
              output1.append(0)
      
      return np.array(output1),FN

In [ ]:
tau=4
keys,non_keys,pred_keys,pred_nonkeys,actual_for_keys,actual_for_non_keys,count1,count2=positive_negative(x_test,y_test,tau,predicted_tst)
keys=np.array(keys)

positive 10000


In [ ]:
print("FNR",count1/len(keys),"FPR",count2/len(non_keys))

FNR 0.1459534698747266 FPR 0.0


In [ ]:
def Test_BF(bloom1, test_data):
  y_pred_bloom = []
  for i in test_data:
    if str(i) in bloom1:
      y_pred_bloom.append(1)
    else:
      y_pred_bloom.append(0)
  y_pred_bloom = np.array(y_pred_bloom)
  return y_pred_bloom

In [ ]:
#creating traditional bloom filter
bloom4 =BloomFilter(max_elements=325000)
dataset=[]
for i in range(len(x_train)):
   dataset.append([x_train[i],y_new_train[i]]) 
for data_point in dataset:
        if data_point[1]==1:
            bloom4.add(data_point[0].all()) 


In [ ]:
Y_pred_bloom_tr=Test_BF(bloom4,x_train)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_new_train,Y_pred_bloom_tr))

0.5


In [ ]:
Y_pred_bloom=Test_BF(bloom4,x_test)

In [ ]:
print(accuracy_score(Y_pred_bloom,y_new_test))

0.5


In [ ]:
#Calculating true negatives
TN=0
for i in range(len(y_new_test)):
  if y_new_test[i] == 0:
       TN+=1
print("TN",TN)
#Calculating true positives
TP=0
for i in range(len(y_new_test)):
  if y_new_test[i] == 1:
       TP+=1
print("TP",TP)

TN 5000
TP 5000


In [ ]:
FN=Test_FNR_bloom(Y_pred_bloom,y_new_test)
if(FN==0):
  print("zero false negative rate")
else:
  print(FN,FN/(FN+TP))
FP=Test_FPR_bloom(Y_pred_bloom,y_new_test)
print(FP,FP/(FP+TN))

zero false negative rate
0
0 0.0


In [ ]:
def Test_FPR_bloom(prediction,y_new_test):
      output1=[]
      count1=0
      for i in range(len(x_test)):
             if(y_new_test[i]==0 and prediction[i]==1):
               count1+=1
      print(count1)
      return count1

In [ ]:
def Test_FNR_bloom(y_pred,y_new_test):
    count1=0
    for i in range(len(y_new_test)):
          if y_new_test[i] == 1 and y_pred[i]==0:
             count1+=1

    if count1==TN:
        return 0
    else:
        return TN-count1